<a href="https://colab.research.google.com/github/DaviAlbini/data-science-projects/blob/main/03-avancados/MortosCampo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predição de Mortos no Campo**

### **Linha de Raciocínio para Predição de `PercentualMortos`**

Para prever o `PercentualMortos`, seguiremos os seguintes passos:

1.  **Entendimento do Problema e dos Dados:**
    *   Revisar o objetivo da predição: prever o percentual de mortalidade no final do ciclo com base em quais informações?
    *   Identificar as variáveis que podem influenciar o `PercentualMortos` (características do lote, manejo, saúde, ambiente, etc.).
    *   Analisar a qualidade dos dados, lidar com valores ausentes ou outliers, se necessário (já fizemos parte disso na análise exploratória).



> *Boa parte já foi tratado no Databricks*



2.  **Preparação dos Dados:**
    *   Selecionar as variáveis (features) que serão usadas para a predição.
    *   Tratar variáveis categóricas (One-Hot Encoding, Label Encoding, etc.).
    *   Normalizar ou escalar variáveis numéricas, se apropriado para o modelo escolhido.
    *   Dividir os dados em conjuntos de treino, validação e teste.

3.  **Engenharia de Features:**
    *   Criar novas features a partir das existentes que possam ter maior poder preditivo (ex: indicadores de saúde acumulados, densidade, etc.).
    *   Incorporar informações temporais, se relevante (ex: características das primeiras semanas de vida).

4.  **Seleção do Modelo:**
    *   Escolher um ou mais algoritmos de machine learning adequados para regressão (já que `PercentualMortos` é uma variável contínua). Opções incluem Regressão Linear, Árvores de Decisão, Random Forest, Gradient Boosting (XGBoost, LightGBM), Redes Neurais, etc.
    *   Considerar a interpretabilidade do modelo versus a performance.

5.  **Treinamento do Modelo:**
    *   Treinar os modelos selecionados nos dados de treino.

6.  **Avaliação do Modelo:**
    *   Avaliar a performance dos modelos nos dados de validação usando métricas apropriadas para regressão (ex: Erro Médio Absoluto - MAE, Erro Quadrático Médio - MSE, Raiz do Erro Quadrático Médio - RMSE, R²).
    *   Realizar validação cruzada para obter uma estimativa mais robusta da performance do modelo.

7.  **Otimização do Modelo:**
    *   Ajustar os hiperparâmetros dos modelos usando técnicas como Grid Search ou Random Search para melhorar a performance.

8.  **Teste Final:**
    *   Avaliar a performance do melhor modelo no conjunto de teste (dados nunca vistos antes) para ter uma estimativa imparcial do desempenho em dados novos.

9.  **Interpretação do Modelo (se aplicável):**
    *   Entender quais features são mais importantes para a predição (Feature Importance).
    *   Analisar como as features influenciam a predição.

10. **Implementação e Monitoramento:**
    *   Planejar como o modelo será usado na prática.
    *   Estabelecer um plano para monitorar a performance do modelo ao longo do tempo e retreiná-lo quando necessário.

Este plano nos guiará no processo de construção do modelo preditivo. Podemos começar pelo passo 2, preparando os dados para o primeiro modelo.

## Bibliotecas e Pacotes

In [1]:
import pandas as pd
import numpy as np
import os
import plotly as plt

## Importação dos Dados

In [3]:
df = pd.read_excel('/content/info_semanais.xlsx')
df.head()

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unidade,Produtor,Lote,Instalacao,DtAlojamento,DtNascimento,Sexo,Categoria,Linhagem,Tecnologia,...,MortosSem5,MortosSem6,EliminadosSem1,EliminadosSem2,EliminadosSem3,EliminadosSem4,EliminadosSem5,EliminadosSem6,PercentualMortos,PercentualEliminados
0,725,0000709677,5700546764,1,2025-02-05,2025-02-05,Misto,FGO PSDO,SRD,Negativa,...,50,83,7,16,21,148,10,0,0.024837,0.009611
1,705,0000826978,5700549837,1,2025-02-12,2025-02-12,Macho,FGO PSDO,ROSS AP95 344,Dark modal,...,162,81,114,92,124,91,69,34,0.029279,0.015736
2,740,0000430001,5700549568,3,2025-02-08,2025-02-08,Macho,FGO PSDO,ROSS AP95 344,Dark modal,...,69,0,0,50,50,50,37,0,0.031967,0.007824
3,732,0000709089,5700550234,2,2025-02-08,2025-02-08,Macho,FGO LV,ROSS AP95 344,Semi Dark,...,0,0,16,14,14,16,0,0,0.020863,0.002353
4,780,0000808147,5700548977,1,2025-02-22,2025-02-22,Macho,FGO PSDO,ROSS AP95 TM4,Positiva,...,291,124,90,56,41,101,0,128,0.047167,0.023111


In [5]:
df.describe()

,Lote,Instalacao,DtAlojamento,DtNascimento,Alojados,Abatidos,peso_1,peso_7,peso_14,peso_21,...,MortosSem5,MortosSem6,EliminadosSem1,EliminadosSem2,EliminadosSem3,EliminadosSem4,EliminadosSem5,EliminadosSem6,PercentualMortos,PercentualEliminados
count,4.210100e+04,42101.000000,42101,40312,40312.000000,33590.000000,40312.000000,37400.000000,36390.000000,34871.000000,...,42101.000000,42101.000000,42101.000000,42101.000000,42101.000000,42101.000000,42101.000000,42101.000000,40312.000000,40312.000000
mean,5.684988e+09,3.401154,2025-04-20 16:09:48.299565568,2025-04-15 21:41:24.064298496,25686.338956,24506.216374,0.043493,0.175925,0.451449,1.090495,...,84.790171,96.514145,87.348400,48.305195,41.669675,35.088881,28.025249,21.489775,0.031643,0.011466
min,1.000000e+00,1.000000,2025-01-01 00:00:00,2024-12-31 00:00:00,220.000000,2463.000000,0.000000,0.000000,0.000000,0.000000,...,-852.000000,-115.000000,0.000000,0.000000,0.000000,0.000000,-205.000000,-1.000000,-0.015297,0.000000
25%,5.700551e+09,1.000000,2025-02-24 00:00:00,2025-02-21 00:00:00,17628.000000,16579.000000,0.040955,0.170000,0.422000,0.850000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015038,0.001645
50%,5.700559e+09,2.000000,2025-04-21 00:00:00,2025-04-16 00:00:00,23000.000000,22042.500000,0.043860,0.180000,0.460000,0.917000,...,28.000000,0.000000,30.000000,21.000000,12.000000,0.000000,0.000000,0.000000,0.025714,0.007264
75%,5.700567e+09,3.000000,2025-06-14 00:00:00,2025-06-07 00:00:00,33000.000000,31182.750000,0.046326,0.190000,0.488000,0.977000,...,105.000000,92.000000,110.000000,59.000000,46.000000,39.000000,22.000000,0.000000,0.041382,0.015556
max,5.700574e+09,80.000000,2025-08-30 00:00:00,2025-08-01 00:00:00,94900.000000,118161.000000,0.066182,2.304000,470.000000,1148.000000,...,12072.000000,17156.000000,8190.000000,3628.000000,54114.000000,5073.000000,5631.000000,4458.000000,0.922727,1.250801
std,2.975302e+08,5.632059,NaN,NaN,10864.910393,10875.179320,0.003671,0.044597,2.463707,15.090315,...,191.799190,261.569588,171.525282,105.638375,279.273627,83.404845,87.455427,93.161418,0.027237,0.016202


## Tratamento dos Dados

## Análise Exploratória

In [6]:
# Análise exploratória do PercentualMortos
print("Estatísticas descritivas para PercentualMortos:")
display(df['PercentualMortos'].describe())

import plotly.express as px

# Histograma para visualizar a distribuição
fig = px.histogram(df, x='PercentualMortos', title='Distribuição de PercentualMortos')
fig.show()

# Box plot para identificar outliers
fig = px.box(df, y='PercentualMortos', title='Box Plot de PercentualMortos')
fig.show()

Estatísticas descritivas para PercentualMortos:


,PercentualMortos
count,40312.000000
mean,0.031643
std,0.027237
min,-0.015297
25%,0.015038
50%,0.025714
75%,0.041382
max,0.922727
